In [ ]:
class Model(nn.Module):
    """Layer comprising a convolution layer followed by LSTM and dense layers."""
    def __init__(
        self,
        seq_len,
        pred_len,
        n_features,
        batch_size,
        hidden,
        nlayers,
        dropout
    ):
        super(Model, self).__init__()
        self.seq_len = seq_len
        self.pred_len=pred_len
        self.hidden=hidden
        self.n_features=n_features
        self.batch_size=batch_size
        self.nlayers=nlayers
        self.dropout = dropout

        self.bn1 = nn.BatchNorm1d(self.n_features)
        self.lstm1 = nn.LSTM(self.n_features, self.hidden, num_layers=self.nlayers, batch_first=True, dropout=self.dropout)
        self.bn2 = nn.BatchNorm1d(self.hidden)
        self.lstm2 = nn.LSTM(self.hidden, self.hidden * 2, num_layers=self.nlayers, batch_first=True, dropout=self.dropout)
        self.bn3 = nn.BatchNorm1d(self.hidden * 2)
        self.lstm3 = nn.LSTM(self.hidden * 2, self.hidden * 4, num_layers=self.nlayers, batch_first=True, dropout=self.dropout)
        self.bn4 = nn.BatchNorm1d(self.hidden * 4)
        self.dense1 = nn.Linear(self.hidden * 4, self.hidden * 2)
        self.bn5 = nn.BatchNorm1d(self.hidden * 2)
        self.dense2 = nn.Linear(self.hidden * 2, self.hidden)
        self.bn6 = nn.BatchNorm1d(self.hidden)
        self.dense3 = nn.Linear(self.hidden, 1)


    def forward(self, inputs):
        """Forward pass.

        Args:
            inputs: torch.Tensor of shape `(batch_size, input_seq_len, num_nodes, in_feat)`

        Returns:
            A tensor of shape `(batch_size, output_seq_len, num_nodes)`.
        """
        inputs=self.bn1(inputs.permute(0,2,1)).permute(0,2,1)
        lstm1_out,_ = self.lstm1(inputs)
        lstm1_out=self.bn2(lstm1_out.permute(0,2,1)).permute(0,2,1)
        lstm2_out,_ = self.lstm2(lstm1_out)
        lstm2_out=self.bn3(lstm2_out.permute(0,2,1)).permute(0,2,1)
        lstm3_out,_ = self.lstm3(lstm2_out)
        lstm3_out=self.bn4(lstm3_out.permute(0,2,1)).permute(0,2,1)
        dense1_out = self.dense1(lstm3_out)
        dense1_out=self.bn5(dense1_out.permute(0,2,1)).permute(0,2,1)
        dense2_out = self.dense2(dense1_out)
        dense2_out=self.bn6(dense2_out.permute(0,2,1)).permute(0,2,1)
        dense3_out = self.dense3(dense2_out)
        dense3_out = dense3_out[:, -self.pred_len:,:]
        return dense3_out